# midiAI

In [90]:
from IPython.display import Audio
from midi2audio import FluidSynth
from music21 import converter, interval, key, meter, note, stream
from pathlib import Path
from pprint import pprint
from tqdm.notebook import tqdm

In [ ]:
def parse(fp):
	s = converter.parse(fp)
	o_key = s.flatten().getElementsByClass(key.KeySignature)[0]
	if (o_key.tonic == "C" and o_key.mode == "major") or (o_key.tonic == "A" and o_key.mode == "minor"):
		return s
	t_key = key.Key("C") if o_key.mode == "major" else key.Key("A", "minor")
	i = interval.Interval(o_key.tonic, t_key.tonic)
	ns = s.transpose(i)
	return ns

def split_long_notes(s):
	ns = stream.Score()
	for p in s.parts:
		np = stream.Part()
		ts = p.recurse().getElementsByClass(meter.TimeSignature).first()
		if ts is None:
			bar_len = 4.0
		else:
			bar_len = ts.barDuration.quarterLength
		current_measure = stream.Measure()
		measure_remaining = bar_len

		for n in p.flat.notesAndRests:
			dur = n.quarterLength
			pitch = n.pitch if isinstance(n, note.Note) else None
			while dur > 0:
				if dur <= measure_remaining:
					n_new = note.Note(pitch, quarterLength=dur) if pitch else note.Rest(quarterLength=dur)
					current_measure.append(n_new)
					measure_remaining -= dur
					dur = 0
				else:
					n_new = note.Note(pitch, quarterLength=measure_remaining) if pitch else note.Rest(quarterLength=measure_remaining)
					current_measure.append(n_new)
					dur -= measure_remaining
					np.append(current_measure)
					current_measure = stream.Measure()
					measure_remaining = bar_len
		if len(current_measure.notes) > 0:
			np.append(current_measure)
		ns.append(np)
	return ns

def gen_notes(s):
	note_measures = []
	leading_measure_last_note = None
	for item in s.parts[1]:
		measure = []
		for n in item:
			n_name = n.pitch.midi if isinstance(n, note.Note) else -1
			n_len = n.quarterLength
			n_info = (n_name, n_len)
			measure.append(n_info)
		if measure[0] == (-1, 0.0):
			leading_fixes = summarize_leading_measure(measure)
			note_measures.append(leading_fixes[0])
			leading_measure_last_note = leading_fixes[1]
		else:
			note_measures.append(measure)
	if leading_measure_last_note:
		note_measures[1].insert(0, leading_measure_last_note)
		for i in range(2, len(note_measures)):
			last = note_measures[i - 1].pop(-1)
			note_measures[i].insert(0, last)
	return note_measures

def summarize_leading_measure(measure):
	new_measure = []
	last_note = None
	leading_measure = measure[0]
	for idx in range(len(measure)):
		if measure[idx] != leading_measure and idx != len(measure) - 1:
			new_measure.append(measure[idx])
		last_note = measure[-1]
	if last_note:
		return (new_measure, last_note)

In [83]:
data_path = Path("data")
files = [f for f in data_path.iterdir() if f.is_file() and f.suffix.lower() == ".midi"]

In [84]:
test = parse("data/500_miles_high.midi")
t = gen_notes(test)

# pprint(t)

In [87]:
test = parse("data/a_cup_of_coffee_a_sandwich_and_you.midi")
t = gen_notes(test)

pprint(t)

[[(55, 1.0), (57, 1.0), (55, 1.0)],
 [(64, 1.0), (60, 1.0), (-1, 1.0), (64, 1.0), (62, 1.0)],
 [(60, 1.0), (-1, 1.0), (64, 1.0), (64, 1.0)],
 [(60, 4.0)],
 [(52, 1.0), (53, 1.0), (52, 1.0), (57, 1.0)],
 [(55, 1.0), (-1, 1.0), (59, 1.0), (57, 1.0)],
 [(55, 1.0), (-1, 1.0), (59, 1.0), (59, 1.0)],
 [(55, 4.0)],
 [(55, 1.0), (57, 1.0), (55, 1.0), (64, 1.0)],
 [(60, 1.0), (-1, 1.0), (64, 1.0), (62, 1.0)],
 [(60, 1.0), (-1, 1.0), (64, 1.0), (64, 1.0)],
 [(60, 4.0)],
 [(52, 1.0), (53, 1.0), (52, 1.0), (57, 1.0)],
 [(55, 1.0), (-1, 1.0), (59, 1.0), (57, 1.0)],
 [(55, 1.0), (-1, 1.0), (57, 1.0), (60, 1.0)],
 [(60, 4.0)],
 [(60, 1.0), (59, 1.0), (58, 1.0), (57, 1.0)],
 [(60, 3.0), (65, 1.0)],
 [(60, 2.0), (62, 1.0), (64, 1.0)],
 [(64, 4.0)],
 [(57, 1.0), (56, 1.0), (55, 1.0), (54, 1.0)],
 [(57, 3.0), (64, 1.0)],
 [(60, 2.0), (64, 1.0), (62, 1.0)],
 [(62, 4.0)],
 [(55, 1.0), (57, 1.0), (55, 1.0), (64, 1.0)],
 [(60, 1.0), (-1, 1.0), (64, 1.0), (62, 1.0)],
 [(60, 1.0), (-1, 1.0), (64, 1.0), (64, 1.

In [35]:
songs = [gen_notes(parse(fp)) for fp in tqdm(files, unit="file")]

  0%|          | 0/153 [00:00<?, ?file/s]

In [ ]:
for s in songs[0]:
	print(s)

[(-1, 0.0), (86, 1.0), (83, 3.0)]
[(83, 1.75), (71, Fraction(2, 3)), (74, Fraction(2, 3)), (81, Fraction(2, 3)), (78, Fraction(1, 3))]
[(78, Fraction(1, 3)), (74, Fraction(2, 3)), (76, 3.0)]
[(76, 2.0), (72, 1.0), (69, Fraction(2, 3)), (65, Fraction(1, 3))]
[(65, Fraction(1, 3)), (62, Fraction(2, 3)), (67, 3.0)]
[(67, 1.0), (65, 3.0)]
[(65, 1.0), (65, 0.5), (64, 1.0), (67, 1.5)]
[(67, 3.75), (65, Fraction(1, 3))]
[(65, Fraction(1, 3)), (64, Fraction(2, 3)), (74, 3.0)]
[(74, 4.0)]
[(72, 1.0), (71, 0.5), (74, 1.5)]
[(74, 3.75), (72, Fraction(1, 3))]
[(72, Fraction(1, 3)), (71, Fraction(2, 3)), (70, 1.0), (68, 0.5), (72, 1.5)]
[(72, 2.0), (-1, 0.5), (67, 0.5), (74, 0.5), (75, 0.5)]
[(68, 0.5), (74, 0.5), (72, 3.0)]
[(72, 4.0)]
[(72, 1.0), (-1, 3.0)]
[(-1, 4.0)]
[(86, 1.0), (83, 3.0)]
[(83, 1.75), (71, Fraction(2, 3)), (74, Fraction(2, 3)), (81, Fraction(2, 3)), (78, Fraction(1, 3))]
[(78, Fraction(1, 3)), (74, Fraction(2, 3)), (76, 3.0)]
[(76, 2.0), (72, 1.0), (69, Fraction(2, 3)), (65, F